<h1 style="font-family:Times New Roman;">Поиск населенных пунктов по части названия в OpenStreerMap</h1>

*Nikolay Sitala 2022, nsitala@ukr.net* 
>Данные по материалам сайта https://ua.transportica.com

In [ ]:
import ipywidgets as widgets
text_ = widgets.Text(
    value='Киев',
    placeholder='Type something',
    description='Поиск подстроки:',
    disabled=False,
    layout = widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)
display(text_)

<h2 style="font-family: 'Times New Roman'">Скрипт поиска</h2>

In [ ]:
import pandas as pd
import panel as pn
import json
import asyncio
import websockets

import nest_asyncio
nest_asyncio.apply()

pn.extension('perspective')


def web_socket():

    payload = {"lang": "ua", "text": text_.value, "country": "UA", "event": "completeCity", "requestNumber": 1}
    data = json.dumps(payload)

    async def hello():
        js = None
        async with websockets.connect("wss://complete.transportica.com:443") as websocket:
            await websocket.send(data)
            return json.loads(await websocket.recv())

    js = asyncio.run(hello())
    p = [[], [], [], [], [], [], []]

    for b in js['data']['result']:
        p[0].append(b['osmId'])
        p[1].append(b['name'])
        p[2].append(b['state'])
        p[3].append(b['lat'])
        p[4].append(b['lon'])
        p[5].append(b['city'])
        
        admin_level = b['hierarchy']
        if len(admin_level) > 1: 
            p[6].append(admin_level[1]['default'])
        else:
            p[6].append(None)
        
    data = {'osmId': p[0], 'city': p[5], 'level': p[6], 'name': p[1], 'state': p[2], 'lat': p[3], 'lon': p[4]}

    return data

data_ = web_socket()
fr = pd.DataFrame.from_dict(data_)
pn.pane.Perspective(fr, width=1000, height=600)